# Проект по SQL

Компания приобретает крупный сервис чтения книг по подписке. Необходимо проанализировать базу данных.

В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

**Цель:** ответить на поставленные вопросы, на основании полученных выводов сделать рекомендацию по новому продукту.

## Загрузка данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

**Таблица books**

Содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

**Таблица authors**

Содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

**Таблица publishers**

Содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства;

**Таблица ratings**

Содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- - book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

**Таблица reviews**

Содержит данные о пользовательских обзорах:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

Схема данных: <https://pictures.s3.yandex.net/resources/scheme_1589269096.png>

## Обзор данных

In [3]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query = ''' SELECT COUNT(DISTINCT book_id),
                    COUNT(DISTINCT author_id)
            FROM books'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count,count
0,1000,636


В таблице по книгам присутсвует 1000 книг и 636 авторов.

In [5]:
query = '''SELECT * FROM authors LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
query = ''' SELECT COUNT(DISTINCT author_id)
            FROM authors'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,636


Значение авторов в таблице authors совпадает с количеством авторов в предыдущей таблице.

In [7]:
query = '''SELECT * FROM publishers LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Загрузка таблицы с названиями издательств.

In [8]:
query = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [9]:
query = ''' SELECT COUNT(DISTINCT username) users,
                    COUNT(DISTINCT rating_id) ratings
            FROM ratings'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,users,ratings
0,160,6456


В данных 160 пользователей и 6456 оценок

In [10]:
query = '''SELECT * FROM reviews LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [11]:
query = ''' SELECT COUNT(DISTINCT username) users,
                    COUNT(DISTINCT review_id) reviews
            FROM reviews'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,users,reviews
0,160,2793


При этом столько же пользователей в таблице с отзывами. Самих отзывов 2793

## Задание 1

Посчитайте, сколько книг вышло после 1 января 2000 года

In [12]:
query = ''' SELECT COUNT(book_id)
            FROM books 
            WHERE publication_date > '2000-01-01'
        '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


Не учитывая первое января, с 01.01.2000 вышло 819 книг.

## Задание 2

Для каждой книги посчитайте количество обзоров и среднюю оценку

In [13]:
query = ''' SELECT b.book_id,
                    b.title,
                    COUNT(DISTINCT re.review_id) review_count,
                    ROUND(AVG(ra.rating), 2) avg_rating
            FROM books AS b
            LEFT JOIN ratings AS ra ON b.book_id=ra.book_id
            LEFT JOIN reviews AS re ON b.book_id=re.book_id
            GROUP BY b.book_id 
            ORDER BY COUNT(re.review_id) DESC
        '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,review_count,avg_rating
0,948,Twilight (Twilight #1),7,3.66
1,750,The Hobbit or There and Back Again,6,4.13
2,673,The Catcher in the Rye,6,3.83
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
4,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
...,...,...,...,...
995,221,Essential Tales and Poems,0,4.00
996,387,Leonardo's Notebooks,0,4.00
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


Книга с самым большим количеством отзывов (7 штук) - Сумерки, имеет рейтинг 3,66. Далее идут книги с 6 и менее отзывами, при этом, имеющие оценку значительно выше первой книги в списке.

## Задание 3

Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [14]:
query = ''' SELECT p.publisher_id,
                    publisher,
                    COUNT(b.book_id)
            FROM books AS b
            LEFT JOIN publishers AS p ON b.publisher_id=p.publisher_id
            WHERE b.num_pages > 50
            GROUP BY p.publisher_id
            ORDER BY COUNT(b.book_id) DESC
            LIMIT 1
        '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher,count
0,212,Penguin Books,42


Самое активное издательство - Penguin Books. Издательство выпустило 42 книги.

## Задание 4

Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [15]:
query = ''' SELECT a.author_id,
                    author,
                    round(AVG(ra.rating), 2) avg_rating
            FROM books AS b
            LEFT JOIN authors AS a ON b.author_id=a.author_id   
            LEFT JOIN ratings AS ra ON b.book_id=ra.book_id
            WHERE b.book_id IN (SELECT book_id
                                FROM ratings
                                GROUP BY book_id
                                HAVING COUNT(rating_id) > 50)
            GROUP BY a.author_id
            ORDER BY AVG(ra.rating) DESC
            LIMIT 1
        '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.29


Самая высокая оценка книг (средняя) у Джоан Роулинг (иллюстратор - Мэри Грандпре). Средняя оценка книг - 4,29.

## Задание 5

Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [16]:
query = ''' SELECT ROUND(AVG(review_count), 2)
            FROM(
            SELECT re.username,
                    COUNT(DISTINCT review_id) review_count
            FROM ratings ra
            JOIN reviews AS re ON re.username = ra.username
            GROUP BY re.username
            HAVING COUNT(DISTINCT rating_id) > 48) users
        '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,round
0,24.0


В среднем пользователи, оставившие 48 оценок, пишут 24 обзора. То есть на каждую оценку приходится два или меньше отзыва.

## Вывод

- Большое количество отзывов не означает высокую оценку книги
- Даже активные пользователи неохотно пишут отзывы
- Самые популярные книги - жанр фентези
- Самое активное издательство - Penguin Books

Таким образом, клиентов может привлечь добавление книг в жанре фентези. Также есть определенная необходимость для поощрения оставления большего количества письменных отзывов среди клиентов.